# Create Sentinel2Dataset

In [21]:
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
import cv2
import numpy as np
import pandas as pd
import torch
import os
import natsort
import glob
from PIL import Image
import natsort
import cv2

In [52]:
DATASET_VERSION = "V2"
TRAIN_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/train/"
VAL_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/val/"
TEST_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/test/"



import os
import pandas as pd

# Constants
DATASET_VERSION = "V2"
BASE_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}"
INPUT_DIR = os.path.join(BASE_DIR, "input")
TARGET_DIR = os.path.join(BASE_DIR, "output")

DATASET_VERSION = "V2"
BASE_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}"
INPUT_DIR = os.path.join(BASE_DIR, "input")
TARGET_DIR = os.path.join(BASE_DIR, "output")
df_input = pd.read_csv(f"{BASE_DIR}/input_l1c.csv", index_col=0)
df_target = pd.read_csv(f"{BASE_DIR}/output_l2a.csv", index_col=0)
base_dir = BASE_DIR

train_target = os.listdir("/mnt/disk/dataset/sentinel-ai-processor/V2/train/target")
# train_input = os.listdir("/mnt/disk/dataset/sentinel-ai-processor/V2/train/input")


df_target["Name"] = df_target["Name"].apply(lambda x:  os.path.basename(x).replace(".SAFE",""))
df_train_target = df_target[df_target["Name"].isin(train_target)]
df_train_target.loc[:, "Name"] = df_train_target["Name"] + ".SAFE"


# Save the sampled data to the corresponding directories
# input_train.to_csv(os.path.join(base_dir, 'train/input.csv'), index=False)
df_train_target.to_csv(os.path.join(base_dir, 'train/target.csv'), index=False)





In [53]:
df_train_target

,id_key,Name,S3Path,Footprint,GeoFootprint,cloud_cover
16,S2A_MSIL1C_20180102T102421_N0500_R065_T32TMT,S2A_MSIL2A_20180102T102421_N0500_R065_T32TMT_2...,/eodata/Sentinel-2/MSI/L2A_N0500/2018/01/02/S2...,geography'SRID=4326;POLYGON ((7.75237634651315...,"{'type': 'Polygon', 'coordinates': [[[7.752376...",68.546486
20,S2A_MSIL1C_20180102T102421_N0500_R065_T32UPU,S2A_MSIL2A_20180102T102421_N0500_R065_T32UPU_2...,/eodata/Sentinel-2/MSI/L2A_N0500/2018/01/02/S2...,geography'SRID=4326;POLYGON ((10.3602792462837...,"{'type': 'Polygon', 'coordinates': [[[10.36027...",94.254524
21,S2A_MSIL1C_20180102T102421_N0500_R065_T32UPV,S2A_MSIL2A_20180102T102421_N0500_R065_T32UPV_2...,/eodata/Sentinel-2/MSI/L2A_N0500/2018/01/02/S2...,geography'SRID=4326;POLYGON ((10.3851737331767...,"{'type': 'Polygon', 'coordinates': [[[10.38517...",99.120772
23,S2A_MSIL1C_20180102T102421_N0500_R065_T32UMU,S2A_MSIL2A_20180102T102421_N0500_R065_T32UMU_2...,/eodata/Sentinel-2/MSI/L2A_N0500/2018/01/02/S2...,geography'SRID=4326;POLYGON ((8.07866651500257...,"{'type': 'Polygon', 'coordinates': [[[8.078666...",59.879202
27,S2A_MSIL1C_20180102T102421_N0500_R065_T32UMV,S2A_MSIL2A_20180102T102421_N0500_R065_T32UMV_2...,/eodata/Sentinel-2/MSI/L2A_N0500/2018/01/02/S2...,geography'SRID=4326;POLYGON ((8.41368654470404...,"{'type': 'Polygon', 'coordinates': [[[8.413686...",79.515254
...,...,...,...,...,...,...
58004,S2C_MSIL1C_20250227T103021_N0511_R108_T31UGR,S2C_MSIL2A_20250227T103021_N0511_R108_T31UGR_2...,/eodata/Sentinel-2/MSI/L2A/2025/02/27/S2C_MSIL...,geography'SRID=4326;POLYGON ((6.21294280425978...,"{'type': 'Polygon', 'coordinates': [[[6.212942...",61.055893
58014,S2C_MSIL1C_20250227T103021_N0511_R108_T31UGP,S2C_MSIL2A_20250227T103021_N0511_R108_T31UGP_2...,/eodata/Sentinel-2/MSI/L2A/2025/02/27/S2C_MSIL...,geography'SRID=4326;POLYGON ((5.68779703536694...,"{'type': 'Polygon', 'coordinates': [[[5.687797...",63.414717
58017,S2B_MSIL1C_20250228T104909_N0511_R051_T31UFP,S2B_MSIL2A_20250228T104909_N0511_R051_T31UFP_2...,/eodata/Sentinel-2/MSI/L2A/2025/02/28/S2B_MSIL...,geography'SRID=4326;POLYGON ((5.04386674293589...,"{'type': 'Polygon', 'coordinates': [[[5.043866...",80.642426
58026,S2B_MSIL1C_20250301T101849_N0511_R065_T32TNS,S2B_MSIL2A_20250301T101849_N0511_R065_T32TNS_2...,/eodata/Sentinel-2/MSI/L2A/2025/03/01/S2B_MSIL...,geography'SRID=4326;POLYGON ((8.99973715756221...,"{'type': 'Polygon', 'coordinates': [[[8.999737...",50.229102


In [22]:
DATASET_VERSION = "V2"
TRAIN_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/train/"
VAL_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/val/"
TEST_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/test/"



def prepare_paths(path_dir):
    
    
    df_input = pd.read_csv(f"{path_dir}/input.csv")
    df_output = pd.read_csv(f"{path_dir}/target.csv")

    df_input["path"] = df_input["Name"].apply(lambda x: os.path.join(path_dir, "input", os.path.basename(x).replace(".SAFE","")))
    df_output["path"] = df_output["Name"].apply(lambda x: os.path.join(path_dir, "input", os.path.basename(x).replace(".SAFE","")))
    
    return df_input, df_output


df_input, df_output =  prepare_paths(TEST_DIR)

In [28]:
def normalize(band, lower_percent=2, upper_percent=98):
    """
    Apply percentile stretching to enhance contrast, only considering valid pixels.
    
    Args:
        band: Input image band as numpy array
        lower_percent: Lower percentile boundary (default 2%)
        upper_percent: Upper percentile boundary (default 98%)
        
    Returns:
        Normalized band with values in [0, 1]
    """
    # Create mask for valid pixels
    valid_mask = (band > 0)
    
    # If no valid pixels, return zeros
    if not np.any(valid_mask):
        return np.zeros_like(band, dtype=np.float32)
    
    # Extract valid pixels for percentile calculation
    valid_pixels = band[valid_mask]
    # Calculate percentiles based only on valid pixels
    lower = np.percentile(valid_pixels, lower_percent)
    upper = np.percentile(valid_pixels, upper_percent)
    
    # Create a copy to avoid modifying the original
    result = band.copy().astype(np.float32)
    
    # Apply stretching only to valid pixels
    result[valid_mask] = np.clip((band[valid_mask] - lower) / (upper - lower), 0, 1)
    
    # Set invalid pixels to 0
    result[~valid_mask] = 0
    
    return result


def read_images(product_paths):

    images = []

    for path in product_paths:

        data = Image.open(path)
        data = np.array(data)
        data = normalize(data)
        
        images.append(data)
        
    
    
    images = np.dstack(images)
    return images

In [ ]:
class Sentinel2Dataset(Dataset):
    def __init__(self, df_x, df_y,
                 train,
                 augmentation,
                 img_size):

        self.df_x = df_x
        self.df_y = df_y
        self.train = train
        self.augmentation = augmentation
        self.img_size = img_size
        # self.transform = get_transforms(train=self.train,
        #                                 augmentation=self.augmentation)

    def __getitem__(self, index):
        # Load images
        x_paths = natsort.natsorted(glob.glob(os.path.join(self.df_x["path"][index], "*.png"), recursive=False))
        x_data = read_images(x_paths)
        x_data = cv2.resize(x_data, (self.img_size, self.img_size), interpolation=cv2.INTER_AREA)
        x_data = torch.from_numpy(x_data).float()
        x_data = torch.permute(x_data, (2, 0, 1))  # HWC to CHW

        y_paths = natsort.natsorted(glob.glob(os.path.join(self.df_x["path"][index], "*.png"), recursive=False))
        y_data = read_images(y_paths)
        y_data = cv2.resize(y_data, (self.img_size, self.img_size), interpolation=cv2.INTER_AREA)
        y_data = torch.from_numpy(y_data).float()
        y_data = torch.permute(y_data, (2, 0, 1))  # HWC to CHW

        # transformed = self.transform(image=x_data, mask=y_data)
        # y_data = transformed["mask"]
        # x_data = transformed["image"]

        return x_data, y_data

    def __len__(self):
        return len(self.df_x)

In [42]:
from torch.utils.data import DataLoader

train_dataset = Sentinel2Dataset(df_x=df_input, df_y=df_output, train=True, augmentation=False, img_size=1024)
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=True,
    num_workers=8,
)